In [1]:
from scipy import optimize
from bingo.evaluation.fitness_function import VectorBasedFunction
import numpy as np
import sys
from numpy import array

In [18]:
number_of_eqps_values = 7
eqps_values_np = np.array([ i*0.002 for i in range(number_of_eqps_values) ])
P_desired = np.array([[1, -0.5, -0.5],
                    [-0.5, 1, -0.5],
                    [-0.5, -0.5, 1]])
dataloc = 'D:\\Work\\software\\st-bingo\\research\\data\\processed_data\\vm_1_bingo_format.txt'
pre_data = np.loadtxt(dataloc)
training_data = []
eqpses = []
for row in pre_data:
    if np.isnan(row[0]) == False:
        training_data.append(row[:3])
        eqpses.append(row[3])
training_data = np.array(training_data)
eqpses = np.array(eqpses)
#print(training_data)

def mapping_(x):

    # mapping = lambda X_0: array([[x[0], x[1], x[2]],
    #    [x[3], x[4], x[5]],
    #    [x[6], x[7], x[8]]]) + (X_0) * (array([[x[9], x[10], x[11]],
    #    [x[12], x[12], x[13]],
    #    [x[14], x[15], x[16]]]))
    mapping = lambda X_0: (x[1]*X_0 + x[0])  * (array([[x[2], x[3], x[4] ],  
        [x[3] , x[6], x[5]],
        [x[4], x[5], x[7]]]))
    mapping = lambda X_0: (x[1]*X_0 + x[0])  * (array([[x[2], 0, 0 ],  
        [0 , x[6], 0],
        [0, 0, x[7]]]))
    
    mapping_matrices = np.array([mapping(e) for e in eqpses])
    P_mapped = mapping_matrices.transpose((0, 2, 1)) @ P_desired @ mapping_matrices
    back_calculated_yield_stresses = []
    for i,stresses in enumerate(training_data):
        #print('here we go', np.transpose(training_data[i]), P_mapped[i], training_data[i])
        back_calculated_yield_stresses.append(P_mapped[i] @ training_data[i])

    error = training_data - back_calculated_yield_stresses
    return error.flatten()



# ((array(0.00500253) + array(0.00500253))/(X_0 + array(0.00500253) + array(0.00500253)) ) * (array([[-0.87240378, -1.01069406, -0.3899734 ],  
#        [-0.8733965 , -1.02045703, -0.37967495],
#        [-0.86253049, -1.02066836, -0.38872487]]))


In [19]:
x0 = [-0.83111601, -0.46657978, -0.06323608, -0.83507571, -0.45586107, -0.0702321, -0.84231627, -0.46026723, -0.05982535, 
      67.83825554, 11.92019618, 22.12241255, 67.75485734, 11.34285823, 22.7518725, 68.42081951, 11.30870116, 22.22108227
]
x0 = [ 0.00500253*2,0.00500253*2,-0.87240378, -1.01069406, -0.3899734, -0.8733965 , -1.02045703, -0.37967495 ]
mapping_(x0)
#results = optimize.minimize(mapping_, )

array([ -33.32840128,   66.65763653,  -33.33213754,  -39.99405264,
         79.98911005,  -39.99855411,  -46.65969601,   93.32056893,
        -46.6649687 ,  -53.32533136,  106.65201312,  -53.33138132,
        -59.99095866,  119.98344257,  -59.99779196,  -66.6565779 ,
        133.31485725,  -66.66420061,  -73.32218905,  146.64625711,
        -73.33060727,   16.36396388,   47.7810875 ,  -64.14860477,
         19.6475585 ,   57.3295353 ,  -76.98137485,   22.93115282,
         66.87797423,  -89.8141415 ,   26.21474684,   76.42640429,
       -102.64690472,   29.49834055,   85.97482544, -115.47966449,
         32.78193396,   95.52323766, -128.3124208 ,   36.06552707,
        105.07164091, -141.14517365,   29.09349721,   37.39508246,
        -66.49195662,   34.91466855,   44.87178773,  -79.79052479,
         40.73583721,   52.34848673,  -93.08908947,   46.5570032 ,
         59.82517944, -106.38765064,   52.3781665 ,   67.30186583,
       -119.68620829,   58.19932712,   74.7785459 , -132.98476

In [20]:
results = optimize.root(mapping_, x0=x0, method='lm', options={"ftol": 1e-16, "xtol": 1e-8, "gtol": 1e-16, "maxiter": 100000})
results

 message: The relative error between two consecutive iterates is at most 0.000000
 success: True
  status: 2
     fun: [-1.387e-07  2.897e-07 ...  1.683e-07 -3.808e-07]
       x: [ 1.227e-01  3.392e-08 -6.653e+00 -1.011e+00 -3.900e-01
           -8.734e-01 -6.653e+00 -6.653e+00]
   cov_x: None
  method: lm
    nfev: 204
    fjac: [[-4.497e+04 -2.165e+02 ...  2.657e-02 -5.315e-02]
           [-2.165e+02 -6.040e+02 ... -9.413e-03  1.883e-02]
           ...
           [ 0.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]
           [ 0.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]]
    ipvt: [1 2 8 7 3 6 4 5]
     qtf: [-5.800e-07  9.951e-07 -2.670e-07 -4.160e-07 -2.648e-06
           -1.280e-07 -9.486e-08  1.990e-07]

In [21]:
np.set_printoptions(threshold=sys.maxsize)
mapping_(results.x)

array([-1.38729341e-07,  2.89695819e-07, -1.37751101e-07, -1.22253617e-07,
        2.59191765e-07, -1.21079708e-07, -9.10373501e-08,  1.99206653e-07,
       -8.96678003e-08, -4.50805899e-08,  1.09740498e-07, -4.35153780e-08,
        1.56167346e-08, -9.20670118e-09,  1.73775945e-08,  9.10545594e-08,
       -1.57634929e-07,  9.30110531e-08,  1.81232906e-07, -3.35544172e-07,
        1.83385055e-07,  6.25884908e-08,  2.07456537e-07, -2.59981547e-07,
        5.34288276e-08,  1.85526801e-07, -2.26880871e-07,  3.70087818e-08,
        1.42482207e-07, -1.65404998e-07,  1.33283748e-08,  7.83227705e-08,
       -7.55538849e-08, -1.76124040e-08, -6.95153801e-09,  4.26724682e-08,
       -5.58135582e-08, -1.13340690e-07,  1.89274004e-07, -1.01275070e-07,
       -2.40844670e-07,  3.64250781e-07,  1.14540434e-07,  1.62280990e-07,
       -2.68706501e-07,  9.88569440e-08,  1.45114527e-07, -2.34233994e-07,
        7.03016312e-08,  1.11414728e-07, -1.70356302e-07,  2.88744957e-08,
        6.11815665e-08, -

In [22]:
results.x

array([ 1.22729873e-01,  3.39206660e-08, -6.65279416e+00, -1.01069406e+00,
       -3.89973400e-01, -8.73396500e-01, -6.65279416e+00, -6.65279416e+00])

In [ ]:
mapping = lambda X_0: array([[-0.83111601, -0.46657978, -0.06323608],
       [-0.83507571, -0.45586107, -0.07023218],
       [-0.84231627, -0.46026723, -0.05982535]]) + (X_0) * (array([[67.83825554, 11.92019618, 22.12241255],
       [67.75485734, 11.34285823, 22.75187253],
       [68.42081951, 11.30870116, 22.22108227]]))
Q = mapping(0)
Q